In [ ]:

# https://towardsdatascience.com/a-simple-way-for-downloading-hundreds-of-clipped-satellite-images-without-retrieving-the-entire-f22d97501fd3/

# https://github.com/microsoft/satclip/blob/main/notebooks/C01_Simple_CSP_Usage.ipynb

# SATCLIP


In [ ]:
!rm -r sample_data .config # Empty current directory
!git clone https://github.com/microsoft/satclip.git . # Clone SatCLIP repository

In [ ]:
!pip install lightning --quiet
!pip install rasterio --quiet
!pip install torchgeo --quiet

In [ ]:
from huggingface_hub import hf_hub_download
import sys
sys.path.append('./satclip') # Add the satclip directory to the system path
from load import get_satclip
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

c = torch.randn(32, 2)  # Represents a batch of 32 locations (lon/lat)

model = get_satclip(
    hf_hub_download("microsoft/SatCLIP-ViT16-L40", "satclip-vit16-l40.ckpt"),
    device=device,
)  # Only loads location encoder by default
model.eval()
with torch.no_grad():
    emb = model(c.double().to(device)).detach().cpu()

In [ ]:
# Save the supply data
survey_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Data_11_5_2025/CH_SINC_foresters_supply_demand_GRID_max_WGS84.csv")
survey_data.drop(columns=['Unnamed: 0'], inplace=True)
survey_data.dropna(subset=['grid_abelow_biom'], inplace=True) # in data, keep only rows where grid_abelow_biom is not NaN
supply_location_embedding_Grid_Max =  survey_data[['lon_WGS84', 'lat_WGS84']]
supply_location_embedding_Grid_Max['Grid_max_index'] = supply_location_embedding_Grid_Max.index
# Create an empty DataFrame with the desired column names directly
emb_df = pd.DataFrame(columns=[f'SATCLIP {i}' for i in range(256)])
# Concatenate the new columns to the right of demand_location_embedding_Grid_Max
supply_location_embedding_Grid_Max = pd.concat([supply_location_embedding_Grid_Max, emb_df], axis=1)

<ipython-input-5-5f79c3cb7358>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  supply_location_embedding_Grid_Max['Grid_max_index'] = supply_location_embedding_Grid_Max.index


In [ ]:
for i in range(len(supply_location_embedding_Grid_Max)):
    c = torch.Tensor([[supply_location_embedding_Grid_Max.iloc[i, 0], supply_location_embedding_Grid_Max.iloc[i, 1]]]) # (lon/lat)
    with torch.no_grad():
        emb = model(c.double().to(device)).detach().cpu()
    supply_location_embedding_Grid_Max.iloc[i,3:] = emb[0,:].numpy()
supply_location_embedding_Grid_Max.to_csv("/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Supply/supply_GRID_max_WGS84_SATCLIP.csv")

In [ ]:
supply_location_embedding_Grid_Max = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Demand/supply_GRID_max_WGS84_SATCLIP.csv")

In [ ]:
demand_location_embedding_Grid_Max

# GEOCLIP

In [ ]:
#   = > THIS IS OK!
!pip install geoclip
from geoclip import LocationEncoder
import torch
import torch.nn as nn
model = LocationEncoder()
c = torch.Tensor([[29.95644245539256,  70.5292325356306]])  #  (lon/lat)

# Change the basedir from '' to '.' to correctly point to the models_visual directory
model.eval()
with torch.no_grad():
  emb = model(c.flip(1).float()).detach().cpu()

import matplotlib.pyplot as plt

plt.hist(emb[0,:].numpy())
plt.show()

In [ ]:
# Save the supply data
survey_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Data_11_5_2025/CH_SINC_foresters_supply_demand_GRID_max_WGS84.csv")
survey_data.drop(columns=['Unnamed: 0'], inplace=True)
survey_data.dropna(subset=['grid_abelow_biom'], inplace=True) # in data, keep only rows where grid_abelow_biom is not NaN
supply_location_embedding_Grid_Max_GEOCLIP =  survey_data[['lon_WGS84', 'lat_WGS84']]
supply_location_embedding_Grid_Max_GEOCLIP['Grid_max_index'] = supply_location_embedding_Grid_Max_GEOCLIP.index
# Create an empty DataFrame with the desired column names directly
emb_df = pd.DataFrame(columns=[f'GEOCLIP {i}' for i in range(512)])
# Concatenate the new columns to the right of demand_location_embedding_Grid_Max
supply_location_embedding_Grid_Max_GEOCLIP = pd.concat([supply_location_embedding_Grid_Max_GEOCLIP, emb_df], axis=1)

<ipython-input-6-92ad9cd25a49>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  supply_location_embedding_Grid_Max_GEOCLIP['Grid_max_index'] = supply_location_embedding_Grid_Max_GEOCLIP.index


In [ ]:
for i in range(len(supply_location_embedding_Grid_Max_GEOCLIP)):
    c = torch.Tensor([[supply_location_embedding_Grid_Max_GEOCLIP.iloc[i, 0], supply_location_embedding_Grid_Max_GEOCLIP.iloc[i, 1]]]) # (lon/lat)
    model.eval()
    with torch.no_grad():
        emb = model(c.flip(1).float()).detach().cpu()
    supply_location_embedding_Grid_Max_GEOCLIP.iloc[i,3:] = emb[0,:].numpy()
supply_location_embedding_Grid_Max_GEOCLIP.to_csv("/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Supply/supply_location_embedding_Grid_Max_GEOCLIP.csv")

# Survey winter & summer embeddings

In [ ]:
!pip install pystac_client odc.stac
!pip install torchgeo

In [ ]:
from pystac_client import Client
from odc.stac import load
import geopy
from geopy.distance import geodesic
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images")  # set WD
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import numpy as np
from pystac_client import Client
from odc.stac import load
import shutil

import os
import tempfile

import time
import http.client
import urllib.parse

import timm
import torch
from lightning.pytorch import Trainer

from torchgeo.datamodules import EuroSAT100DataModule
from torchgeo.models import ResNet18_Weights
from torchgeo.trainers import ClassificationTask

import matplotlib.pyplot as plt # Import matplotlib.pyplot with the alias plt

import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.models import resnet50, ResNet50_Weights
# from torchvision.models import resnet18, ResNet18_Weights # Remove duplicate import
from torchgeo.models import ResNet18_Weights

import os
import numpy as np
import glob # Import glob for moving files
import timm # Import timm
import pandas as pd
from sklearn.cluster import KMeans
import pickle

from humanfriendly import format_timespan



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# This option is to get images from AWS
# This has more images than Google, but some of them are partial / corrupted, so you'd have to filter those out

client=Client.open("https://earth-search.aws.element84.com/v1")
collection= "sentinel-2-l2a"
date="2020-01-01/2021-12-31" # This is the period  / year when the survey was collected
cloud_cover= (0, 0.1) # ("min_cloud_cover", "max_cloud_cover") 0.1 is 10% cloud cover

# First coordinate is bottom-left, second one is top-right


def save_images(client, collection, date, cloud_cover, start_coords, data_index):
    new_path = os.path.join(basic_path, str(data_index))
    # Check if directory exists before creating
    if not os.path.exists(new_path):
        os.makedirs(new_path)
    os.chdir(new_path)  # set WD

    point_south = geodesic(meters=500).destination(start_coords, bearing=180)
    point1_coords = geodesic(meters=500).destination(point_south, bearing=270) # bottom-left
    point_north = geodesic(meters=500).destination(start_coords, bearing=0)
    point2_coords = geodesic(meters=500).destination(point_north, bearing=90) # top-right

    # bbox= [-120.15,38.93,  -119.88,39.25] # Lake Tohoe Region # first longitude and then latitude  # Coordinates from my file work
    bbox = [point1_coords.longitude, point1_coords.latitude, point2_coords.longitude, point2_coords.latitude]

    print(f"Searching for items for data_index {data_index} with bbox: {bbox}") # Print bbox

    search = client.search(collections=[collection], bbox=bbox , datetime=date, query=[
            f"eo:cloud_cover<{cloud_cover[1]}",
            f"eo:cloud_cover>{cloud_cover[0]}"
        ])

    items = list(search.items()) # Convert to list to easily check length
    num_items = len(items)
    print(f"Number of items found: {num_items}") # Print number of items found

    if num_items == 0:
        print(f"No STAC items found for data_index {data_index}. Skipping.")
        # Change back to the basic_path before continuing the loop
        os.chdir(basic_path)
        return # Skip processing for this data point

    # If items are found, proceed with loading and processing
    data =load(items, bbox=bbox , groupby = "solar_day", chunks ={})
    numb_days= data[["red"]].time.size

    if numb_days == 0:
         print(f"No valid data layers loaded for data_index {data_index}. Skipping.")
         # Change back to the basic_path before continuing the loop
         os.chdir(basic_path)
         return # Skip processing for this data point

    for t in range(0,numb_days):
        # Ensure the data has the expected bands before accessing
        if "red" in data.data_vars and "green" in data.data_vars and "blue" in data.data_vars:
            rgb_image = data[["red","green","blue"]].isel(time=t).to_array()
            dt = pd.to_datetime(rgb_image.time.values)

            year = dt.year
            month = dt.month
            day = dt.day

            # Double digit for month and day
            month_str = f'{month:02d}'
            day_str = f'{day:02d}'

            date_string= f"{year}_{month_str}_{day_str}"
            date_string_title= f"{month_str}/{day_str}/{year}"

            # Check if the image has valid data before calculating coverage
            if rgb_image.size > 0:
                # Assuming the second channel (index 1) corresponds to the green band
                count1 = np.count_nonzero(~np.isnan(rgb_image[1,:,:])) # Count non-NaN pixels in green band
                count2 = np.size(rgb_image[1,:,:])
                # Avoid division by zero
                ratio = count1 / count2 if count2 > 0 else 0

                # Print Date and Coverage
                print(f"Date:{date_string}, Coverage:{format(ratio*100,'.2f')}%")

                if ratio>0.7:
                    # Plot the RGB image
                    fig, ax = plt.subplots()
                    rgb_image.plot.imshow(robust=True, ax=ax)
                    ax.set_axis_off() # Remove axes
                    plt.subplots_adjust(left=0, right=1, top=1, bottom=0) # Remove whitespace around the plot
                    plt.savefig(f"rgb_image_{date_string}.png", pad_inches=0) # Save without padding
                    plt.close(fig)
            else:
                print(f"Image for date {date_string} is empty. Skipping.")
        else:
            print(f"RGB bands not found in data for date {dt}. Skipping.")

    # Change back to the basic_path after processing a data point
    os.chdir(basic_path)



def get_embeddings(image_folder):
    # Define a new subdirectory for all images
    single_class_folder = os.path.join(image_folder, 'all_images')
    # Create the subdirectory if it doesn't exist
    os.makedirs(single_class_folder, exist_ok=True)

    # Move all image files from the original folder to the new subdirectory
    # You might need to adjust the image extensions based on your files
    for img_path in glob.glob(os.path.join(image_folder, '*.jpg')) + \
                    glob.glob(os.path.join(image_folder, '*.png')): # Add other extensions if needed
        # Check if the file is already in the target directory before attempting to move
        if os.path.isfile(img_path): # Ensure it's a file before attempting to move
            if not os.path.exists(os.path.join(single_class_folder, os.path.basename(img_path))):
                os.rename(img_path, os.path.join(single_class_folder, os.path.basename(img_path)))
    # Define the transformations to apply to the images
    # You might need to adjust these based on the expected input size of your model
    # Use the mean and std from the weights metadata if available, otherwise use common defaults
    weights = ResNet18_Weights.SENTINEL2_RGB_MOCO
    mean = weights.meta.get('mean', [0.485, 0.456, 0.406])
    std = weights.meta.get('std', [0.229, 0.224, 0.225])
    input_size = weights.meta.get('input_size', (3, 640, 480)) # 640x480 is the original, recommendation is  224, 224

    transform = transforms.Compose([
        transforms.Resize((input_size[1], input_size[2])), # Use input_size from weights
        transforms.ToTensor(),        # Convert images to PyTorch tensors
        transforms.Normalize(mean=mean, std=std) # Use mean and std from weights
    ])
    # Load the dataset using the new subdirectory
    # Ensure ImageFolder points to the directory containing the class subdirectories ('all_images')
    # Check if the single_class_folder exists and is not empty before creating ImageFolder
    if not os.path.exists(single_class_folder) or not os.listdir(single_class_folder):
        print(f"No images found in {single_class_folder}. Skipping embedding generation.")
        return None, None

    dataset = ImageFolder(image_folder, transform=transform)
    # Create a DataLoader
    dataloader = DataLoader(dataset, batch_size=8, shuffle=False) # Set shuffle=False to maintain order
    # Load a pre-trained model using timm and then load the state dictionary from torchgeo weights
    model = timm.create_model('resnet18', pretrained=True, in_chans=weights.meta['in_chans'])
    model.load_state_dict(weights.get_state_dict(progress=True), strict=False)
    # Remove the classification layer to get the embedding vectors
    model = torch.nn.Sequential(*(list(model.children())[:-1]))
    # Set the model to evaluation mode
    model.eval()
    # Get the embedding vectors
    embeddings = []
    image_paths = []
    with torch.no_grad():
        for inputs, _ in dataloader:
            outputs = model(inputs)
            # Squeeze the output to remove the extra dimension
            embedding_batch = outputs.squeeze().cpu().numpy()
            # Add a batch dimension if the squeeze operation removed it (e.g., for a batch of 1)
            if embedding_batch.ndim == 1:
                embedding_batch = np.expand_dims(embedding_batch, axis=0)
            embeddings.append(embedding_batch)

    # After the loop, collect the image paths from the dataset samples
    image_paths = [sample[0] for sample in dataset.samples]
    # Concatenate the embeddings
    if embeddings: # Check if the list is not empty before concatenating
        embeddings = np.concatenate(embeddings, axis=0)
    else:
        return None, None # Return None if no embeddings were generated

    cleaned_image_paths = [path.split('/')[-1] for path in image_paths]# in each element of  image_paths, remove all before last /
    cleaned_image_paths = [path.split('/')[-1].split('.')[0][-5:] for path in cleaned_image_paths] # Apply split and slice to the cleaned paths
    processed_months = []
    for path in cleaned_image_paths:
        try:
            month = int(path[:2]) # Try to convert the first two characters to an integer
            processed_months.append(month)
        except ValueError:
            print(f"Warning: Could not extract month from filename part '{path}'. Skipping.")
            # Optionally, you could append a placeholder like None or -1 to indicate an invalid month
            processed_months.append(None) # Or handle as appropriate for your data

    # Filter out None values before calculating summer_deviations
    valid_months = [month for month in processed_months if month is not None]

    if valid_months: # Check if the list is not empty before proceeding
        summer_deviations = [month - 7 for month in valid_months]
    else:
        summer_deviations = None # Or handle as an error

    return embeddings, summer_deviations


def get_combined_vectors(embeddings, summer_deviations):
    # Check if there's only one embedding
    if embeddings.shape[0] == 1:
        # If only one embedding, duplicate it to match the expected output dimension
        print("Only one embedding found. Duplicating the vector.")
        combined_vectors = np.concatenate((embeddings[0], embeddings[0]))
    else:
        # Proceed with KMeans if there are two or more embeddings
        kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
        # Fit KMeans to the embeddings
        kmeans.fit(embeddings)
        cluster_labels = kmeans.labels_
        cluster_centers = kmeans.cluster_centers_
        zero_indices = [i for i, label in enumerate(cluster_labels) if label == 0]
        one_indices = [i for i, label in enumerate(cluster_labels) if label == 1]
        # Handle cases where a cluster might be empty due to KMeans result (unlikely with only two samples and n_clusters=2, but good practice)
        if not zero_indices:
            median_vector_cluster_zero = np.zeros(embeddings.shape[1]) # Or handle as an error
        else:
            median_vector_cluster_zero = np.median(embeddings[zero_indices, :], axis=0)

        if not one_indices:
            median_vector_cluster_one = np.zeros(embeddings.shape[1]) # Or handle as an error
        else:
             median_vector_cluster_one = np.median(embeddings[one_indices, :], axis=0)
        summer_deviations_cluster_zero = [summer_deviations[i] for i in zero_indices]
        summer_deviations_cluster_zero = np.array(summer_deviations_cluster_zero)
        summer_deviations_cluster_one = [summer_deviations[i] for i in one_indices]
        summer_deviations_cluster_one = np.array(summer_deviations_cluster_one)
        # Compare means only if clusters are not empty
        if len(summer_deviations_cluster_zero) > 0 and len(summer_deviations_cluster_one) > 0:
            if summer_deviations_cluster_zero.mean() < summer_deviations_cluster_one.mean():
                combined_vectors = np.hstack((median_vector_cluster_zero, median_vector_cluster_one))
            else:
                combined_vectors = np.hstack((median_vector_cluster_one, median_vector_cluster_zero))
        elif len(summer_deviations_cluster_zero) > 0:
            # Only cluster 0 has samples, duplicate its median
             combined_vectors = np.hstack((median_vector_cluster_zero, median_vector_cluster_zero))
        elif len(summer_deviations_cluster_one) > 0:
            # Only cluster 1 has samples, duplicate its median
             combined_vectors = np.hstack((median_vector_cluster_one, median_vector_cluster_one))
        else:
            # Should not happen if embeddings has more than one sample, but as a safeguard
            combined_vectors = np.zeros(embeddings.shape[1] * 2) # Return zero vector or handle as error
    return combined_vectors


def send_push_final(total_elapsed_time):
    token = 'YYY'
    user = 'ZZZ'
    conn = http.client.HTTPSConnection("api.pushover.net:443")
    conn.request("POST", "/1/messages.json",
      urllib.parse.urlencode({
        "token": token,
        "user": user,
        "title": "Job done - images downloaded",
        "message": f"Total elapsed time = {total_elapsed_time}",
      }), { "Content-type": "application/x-www-form-urlencoded" })
    conn.getresponse()

def format_timespan(seconds):
    """Formats a timespan in seconds into a human-readable string."""
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    return f"{int(hours)}h {int(minutes)}m {seconds:.2f}s"

In [ ]:
# survey_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Data_11_5_2025/CH_data_demand_14_4_GRID_max.csv")
# survey_data.drop(columns=['Unnamed: 0'], inplace=True)
# survey_data.dropna(subset=['grid_abelow_biom'], inplace=True) # in data, keep only rows where grid_abelow_biom is not NaN

In [ ]:
# GETTING EMBEDDINGS FOR DATA FROM THE SURVEY

# lOAD THE DATA
survey_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Data_11_5_2025/CH_SINC_foresters_supply_demand_GRID_max_WGS84.csv")
survey_data.drop(columns=['Unnamed: 0'], inplace=True)
survey_data.dropna(subset=['grid_abelow_biom'], inplace=True) # in data, keep only rows where grid_abelow_biom is not NaN

# IMPLEMENT THE LOOP
for data_index in range(len(survey_data)):
    start_coords = (survey_data['lat_WGS84'][survey_data.index[data_index]], survey_data['lon_WGS84'][survey_data.index[data_index]])
    save_images(client, collection, date, cloud_cover, start_coords, survey_data.index[data_index])
os.chdir(basic_path)

# Order subfolders by name / number, in increasing order
subfolders = [f.path for f in os.scandir(basic_path) if f.is_dir()]
def extract_number(folder_path):
  return int(folder_path.split('/')[-1])
subfolders.sort(key=extract_number)
# Delete the last subfolder
if subfolders:
  last_subfolder = subfolders[-1]
  shutil.rmtree(last_subfolder)
  print(f"Deleted: {last_subfolder}")
  # Update the subfolders list
  subfolders = subfolders[:-1]
else:
  print("No subfolders to delete.")


# REMOVE ALL IMAGES THAT ARE JUST BLACK OR WHITE!!!!! i.e. smaller than 15kb
for subfolder_name in os.listdir(basic_path):
  subfolder_path = os.path.join(basic_path, subfolder_name)
  if os.path.isdir(subfolder_path):
    for filename in os.listdir(subfolder_path):
      file_path = os.path.join(subfolder_path, filename)
      if os.path.isfile(file_path):
        if os.path.getsize(file_path) < 15 * 1024:  # 15 KB in bytes
          print(f"Removing file: {file_path} (size: {os.path.getsize(file_path)} bytes)")
          # os.remove(file_path)

#  within basic_path, make a list of all of the sub-folders that are either empty or have just one file in them. Then emtpy all of these sub-folders

empty_or_single_file_subfolders = []
for subfolder_name in os.listdir(basic_path):
  subfolder_path = os.path.join(basic_path, subfolder_name)
  if os.path.isdir(subfolder_path):
    files_in_subfolder = [f for f in os.listdir(subfolder_path) if os.path.isfile(os.path.join(subfolder_path, f))]
    if len(files_in_subfolder) <= 1:
      empty_or_single_file_subfolders.append(subfolder_path)

print("Subfolders to be emptied:")
for folder in empty_or_single_file_subfolders:
  print(folder)

# Empty the identified subfolders
for subfolder_path in empty_or_single_file_subfolders:
  print(f"Emptying subfolder: {subfolder_path}")
  for filename in os.listdir(subfolder_path):
    file_path = os.path.join(subfolder_path, filename)
    if os.path.isfile(file_path):
      os.remove(file_path)
      print(f"Removed file: {file_path}")

print("Finished emptying subfolders.")

# from empty_or_single_file_subfolders, remove '/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Demand/GRID_max/' from each entry
# Then repeat the loop
empty_or_single_file_subfolders_index = [folder.replace('/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Demand/GRID_max/', '') for folder in empty_or_single_file_subfolders]

for data_index in empty_or_single_file_subfolders_index:
    start_coords = (survey_data.loc[int(data_index), 'survey_Q9_latitude'], survey_data.loc[int(data_index), 'survey_Q9_longitude'])
    save_images(client, collection, date, cloud_cover, start_coords, int(data_index)) # Pass data_index as integer to save_images
os.chdir(basic_path)

# Seems that it's correct... some points (cca 600 in total) really don't have images, or have just one. Then proceed with analysis

In [ ]:
# remove empty sub-folders in basic_path

# After processing all folders, remove any remaining empty subfolders
for subfolder_name in os.listdir(basic_path):
  subfolder_path = os.path.join(basic_path, subfolder_name)
  if os.path.isdir(subfolder_path):
    # List all items (files and subfolders) in the directory
    items_in_subfolder = os.listdir(subfolder_path)
    # If the directory is empty
    if not items_in_subfolder:
      print(f"Removing empty subfolder: {subfolder_path}")
      os.rmdir(subfolder_path) # Use rmdir for empty directories


# order ascending list_of_folders

all_entries = os.listdir(basic_path)

# Filter to keep only subdirectories that can be converted to integers
list_of_folders = [entry for entry in all_entries if os.path.isdir(os.path.join(basic_path, entry)) and entry.isdigit()]

# Sort the filtered list by converting to integer
list_of_folders.sort(key=lambda x: int(x))

###   GET JUST EMBEDDINGS AND SAVE AS A DICTIONARY - of only folders that are not empty

total_start = time.time()

results_dict = {}

# Define the starting index for the loop
start_loop_index = 0  # Replace with the desired starting value of i

for i in range(start_loop_index, len(list_of_folders)):
    # Construct the image folder path
    index  = list_of_folders[i]
    image_folder = f'/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Supply/Working_GRID_max_2/{index}'

    # Check if the image folder exists
    if not os.path.exists(image_folder):
        print(f"Image folder not found for index {i}: {image_folder}. Skipping.")
        continue # Skip to the next iteration if the folder doesn't exist

    # Get embeddings and summer deviations
    embeddings, summer_deviations = get_embeddings(image_folder)

    # Store results in the dictionary
    if embeddings is not None and summer_deviations is not None:
        results_dict[index] = {
            'embeddings': embeddings,
            'summer_deviations': summer_deviations
        }
    else:
        results_dict[index] = {
            'embeddings': 0,
            'summer_deviations': 0
        }
        print(f"Could not generate embeddings for index {index}")

with open('embeddings_supply_full.pkl', 'wb') as f:
    pickle.dump(results_dict, f)

total_end = time.time()
total_elapsed_seconds = total_end - total_start
total_elapsed_time = format_timespan(total_elapsed_seconds)
send_push_final(total_elapsed_time)

# or demand
# with open('embeddings_demand_full.pkl', 'rb') as f:
#     loaded_dict = pickle.load(f)

# Open the file to proceed
with open('/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Demand/embeddings_demand_full.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)


# Now get the clustered embeddings and put them in a DF, where the Grid_max_index matches the indexes of the Grid_max_index demand data file
# First embedding  / first 512 are the more summer, then more winter images
total_start = time.time()

for i in range(1024):
    col_name = f'image_vector_{i}'
    if col_name not in images_vectors.columns:
        images_vectors[col_name] = np.nan # Initialize with NaN

# Iterate through the index of the dataframe
for i in range(len(images_vectors)):
    # Construct the image folder path
    index  = images_vectors['Grid_max_index'].values[i]
    try:
        one_point_embeddings = loaded_dict[str(index)]
        combined_vectors = get_combined_vectors(one_point_embeddings['embeddings'], one_point_embeddings['summer_deviations'])
        if len(combined_vectors) == 512:
            combined_vectors = np.concatenate((combined_vectors, combined_vectors))
        elif len(combined_vectors) != 1024:
            print(f"Unexpected combined_vectors length for index {i}: {len(combined_vectors)}. Skipping.")
            # Mark the row with NaN if the length is not 1024 after duplication attempt
            images_vectors.loc[i, [f'image_vector_{j}' for j in range(1024)]] = np.nan
            continue # Skip to the next iteration if the length is still not 1024
        images_vectors.loc[i, [f'image_vector_{j}' for j in range(1024)]] = combined_vectors
    except KeyError:
        print(f"No embeddings generated for index {i}. Marking row with NaN.")
        images_vectors.loc[i, [f'image_vector_{j}' for j in range(1024)]] = None

print("Processing complete.")

images_vectors.to_csv('/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/demand_images_vectors_embeddings.csv')

total_end = time.time()
total_elapsed_seconds = total_end - total_start
total_elapsed_time = format_timespan(total_elapsed_seconds)
send_push_final(total_elapsed_time)

# open embeddings and make final vectors for the supply as based on winter / summer images

import pickle
with open('/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Supply/embeddings_supply_full.pkl', 'rb') as f:
    data = pickle.load(f)
list_of_keys = list(data.keys())

# Make an empty DF
df = pd.DataFrame(np.nan, index=range(1025), columns=['Grid_Max_Index'] + [f'embedding_{i}' for i in range(1024)])

for i in range(len(list_of_keys)):
    try:
        combined_vectors = get_combined_vectors(data[list_of_keys[i]]['embeddings'], data[list_of_keys[i]]['summer_deviations'])
        if combined_vectors is not None and len(combined_vectors) == 1024:
            df.loc[i, [f'embedding_{j}' for j in range(1024)]] = combined_vectors
            df.loc[i, 'Grid_Max_Index'] = int(list_of_keys[i])
        else:
            df.loc[i, 'Grid_Max_Index'] = int(list_of_keys[i])
            df.loc[i, [f'embedding_{j}' for j in range(1024)]] = np.nan
    except IndexError:
        # Handle cases where the index i might be out of bounds for list_of_keys
        print(f"IndexError: list_of_keys index {i} is out of bounds.")
        df.loc[i, 'Grid_Max_Index'] = np.nan # Assign NaN if index is out of bounds
        df.loc[i, [f'embedding_{j}' for j in range(1024)]] = np.nan
    except Exception as e:
        # Handle other potential errors during processing
        print(f"An error occurred at index {i}: {e}")
        df.loc[i, 'Grid_Max_Index'] = int(list_of_keys[i]) # Still try to set index if possible
        df.loc[i, [f'embedding_{j}' for j in range(1024)]] = np.nan

df.to_csv('/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Supply/supply_embeddings_winter_summer.csv')


# Alternative version to get the embeddings V1

# Add columns for the combined vectors if they don't exist
embedding_dim = combined_vectors.shape[0]
for i in range(embedding_dim):
    col_name = f'combined_vector_{i}'
    if col_name not in images_vectors.columns:
        images_vectors[col_name] = np.nan

# Loop and populate the new columns
for i in range(len(images_vectors)):
    # Assuming 'Grid_max_index' is the column in images_vectors that corresponds
    # to the folder name (which is the data index)
    data_index = images_vectors['Grid_max_index'].iloc[i]
    new_path = os.path.join(basic_path, str(int(data_index))) # Ensure data_index is treated as int for path

    # Check if the directory exists and contains images
    if os.path.isdir(new_path):
      image_files = [f for f in os.listdir(os.path.join(new_path, 'all_images')) if os.path.isfile(os.path.join(new_path, 'all_images', f))]
      if len(image_files) > 1: # Need at least two images for KMeans with 2 clusters
          try:
              embeddings = get_embeddings(new_path)
              if embeddings is not None and embeddings.shape[0] > 1: # Ensure embeddings are calculated and there are at least 2
                  combined_vectors = get_combined_vectors(embeddings)
                  # Ensure combined_vectors has the expected dimension
                  if combined_vectors.shape[0] == embedding_dim:
                      # Assign the combined vector elements to the corresponding columns in the current row
                      for j in range(embedding_dim):
                          images_vectors.loc[images_vectors.index[i], f'combined_vector_{j}'] = combined_vectors[j]
                  else:
                      print(f"Warning: Combined vector dimension mismatch for index {images_vectors.index[i]}. Expected {embedding_dim}, got {combined_vectors.shape[0]}. Skipping.")
              else:
                  print(f"Warning: Insufficient embeddings or failed to get embeddings for index {images_vectors.index[i]}. Skipping.")
          except Exception as e:
              print(f"Error processing index {images_vectors.index[i]}: {e}")
      else:
          print(f"Skipping index {images_vectors.index[i]}: Not enough images in folder {new_path}.")
    else:
      print(f"Skipping index {images_vectors.index[i]}: Folder {new_path} does not exist or is not a directory.")




# Alternative version to get the embeddings V2

# Add new columns for the combined vectors
# Assuming combined_vectors will have 2 rows (median vectors for two clusters)
# And the number of columns will be the dimension of the embedding vectors
# Let's determine the dimension by processing one folder first
# example_folder = [f.path for f in os.scandir(basic_path) if f.is_dir() and len([name for name in os.listdir(f.path) if os.path.isfile(os.path.join(f.path, name))]) > 1][0] # Find a folder with more than 1 file
new_path = os.path.join(basic_path, str(images_vectors['Grid_max_index'].iloc[0]))
embeddings = get_embeddings(new_path)
combined_vectors = get_combined_vectors(embeddings)
embedding_dim = combined_vectors.shape[0]
for i in range(embedding_dim):
    images_vectors[f'combined_vec1_{i}'] = np.nan
    images_vectors[f'combined_vec2_{i}'] = np.nan

# Need to handle the case where combined_vectors has only one row (when KMeans finds only one cluster or there is only one image)
# In that case, we assign the same vector to both combined_vec1 and combined_vec2

for i in range(len(images_vectors)):
    current_index = images_vectors.index[i] # Get the actual index from the survey_data/images_vectors DataFrame
    new_path = os.path.join(basic_path, str(images_vectors['Grid_max_index'].iloc[0]))
    # Check if the directory exists and has more than one file before processing
    if os.path.isdir(new_path) and len([name for name in os.listdir(new_path) if os.path.isfile(os.path.join(new_path, name))]) > 1:
        embeddings = get_embeddings(new_path)
        combined_vectors = get_combined_vectors(embeddings)

        if combined_vectors.shape[0] == 2:
            # Assign the two median vectors to the respective columns
            images_vectors.loc[current_index, [f'combined_vec1_{j}' for j in range(embedding_dim)]] = combined_vectors[0, :]
            images_vectors.loc[current_index, [f'combined_vec2_{j}' for j in range(embedding_dim)]] = combined_vectors[1, :]
        elif combined_vectors.shape[0] == 1:
            # Assign the single median vector to both sets of columns
            images_vectors.loc[current_index, [f'combined_vec1_{j}' for j in range(embedding_dim)]] = combined_vectors[0, :]
            images_vectors.loc[current_index, [f'combined_vec2_{j}' for j in range(embedding_dim)]] = combined_vectors[0, :]
        else:
            # Handle cases with no valid combined vectors (shouldn't happen if embeddings are generated, but as a safeguard)
            print(f"Warning: No valid combined vectors for index {current_index}")

    elif os.path.isdir(new_path) and len([name for name in os.listdir(new_path) if os.path.isfile(os.path.join(new_path, name))]) == 1:
        # Handle case with exactly one image file
        embeddings = get_embeddings(new_path) # This will give one embedding vector
        # Assign this single vector to both combined_vec1 and combined_vec2
        images_vectors.loc[current_index, [f'combined_vec1_{j}' for j in range(embedding_dim)]] = embeddings[0, :]
        images_vectors.loc[current_index, [f'combined_vec2_{j}' for j in range(embedding_dim)]] = embeddings[0, :]

    else:
        # Handle cases where the directory does not exist or has no image files
        print(f"Skipping index {current_index}: Directory not found or no image files")
        # The combined vector columns will remain NaN for this row

print("Finished processing images and adding combined vectors.")
print(images_vectors.head()) # Display the first few rows of the updated DataFrame

# full data - winter & summer embeddings

In [ ]:
!pip install humanfriendly

In [ ]:
#### HOW TO GET IMAGES FROM GOOGLE ENGINE
# This is code to get winter and summer images for each of the point-locations in the full data for all of Europe's forests
# Data is split into 4 chunks, as it was run on 4 paralel kernels for a couple of weeks
# This is example from the first one

import ee
import geopandas as gpd
from IPython.display import Image # Import the Image class
import requests
import datetime
import json

from torchvision.models.resnet import ResNet18_Weights+
from PIL import Image, UnidentifiedImageError # Import Image and UnidentifiedImageError
from humanfriendly import format_timespan


from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Full_grid/Grid_Max/Chunk_1")  # set WD

data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Data_11_5_2025/full_data_WGS84_chunks/chunk_1.csv")
basic_path = "/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Full_grid/Grid_Max/Chunk_1"

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='subtle-harmony-410811')


def get_images(basic_path, data_index, latitude, longitude, year):
    new_path = os.path.join(basic_path, str(data_index))
    # Check if directory exists before creating
    if not os.path.exists(new_path):
        os.makedirs(new_path)
    os.chdir(new_path)  # set WD

    point = ee.Geometry.Point(longitude, latitude)
    buffer = point.buffer(1000 / 2) # Create a square bbox by buffering by half the size

    for month in range(5, 9): # Loop through all months of the year
        start_date = ee.Date(f'{year}-{month}-01')
        # Calculate end date for the month, handling month lengths and leap years
        end_date = start_date.advance(1, 'month')

        # Define the Sentinel-2 image collection
        collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
            .filterDate(start_date, end_date) \
            .filterBounds(buffer) \
            .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10) \
            .map(lambda image: image.select(['B4', 'B3', 'B2']).divide(10000)) # Select RGB bands and scale

        # Get a single image (median composite) for the month
        image = collection.median()

        if image and image.bandNames().size().getInfo() > 0: # Check if an image was found and has bands
            # Define visualization parameters for True Color (RGB)
            vis_params_true_color = {
                'min': 0.0,
                'max': 0.3,
            }

            # Get the thumbnail URL
            try:
                thumb_url = image.getThumbUrl({
                    'bands': ['B4', 'B3', 'B2'],
                    'min': vis_params_true_color['min'],
                    'max': vis_params_true_color['max'],
                    'region': buffer.getInfo(),
                    'dimensions': 512
                })

                # Download the image
                response = requests.get(thumb_url)
                if response.status_code == 200:
                    month_str = str(month).zfill(2) # Pad month with leading zero
                    filename = f"image_{year}_{month_str}.png"
                    filepath = os.path.join(new_path, filename) # Save in the new_path directory
                    with open(filepath, 'wb') as f:
                        f.write(response.content)
                    print(f"Saved {filename} to {new_path}")
                    break
                else:
                    print(f"Error downloading image for {year}-{month}: {response.status_code}")
            except ee.EEException as e:
                print(f"Earth Engine error for {year}-{month}: {e}")
        else:
            print(f"No suitable image found for {year}-{month}")

    for month in (1,2,3,4,10,11,12): # Loop through all months of the year
        start_date = ee.Date(f'{year}-{month}-01')
        # Calculate end date for the month, handling month lengths and leap years
        end_date = start_date.advance(1, 'month')

        # Define the Sentinel-2 image collection
        collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
            .filterDate(start_date, end_date) \
            .filterBounds(buffer) \
            .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10) \
            .map(lambda image: image.select(['B4', 'B3', 'B2']).divide(10000)) # Select RGB bands and scale

        # Get a single image (median composite) for the month
        image = collection.median()

        if image and image.bandNames().size().getInfo() > 0: # Check if an image was found and has bands
            # Define visualization parameters for True Color (RGB)
            vis_params_true_color = {
                'min': 0.0,
                'max': 0.3,
            }

            # Get the thumbnail URL
            try:
                thumb_url = image.getThumbUrl({
                    'bands': ['B4', 'B3', 'B2'],
                    'min': vis_params_true_color['min'],
                    'max': vis_params_true_color['max'],
                    'region': buffer.getInfo(),
                    'dimensions': 512
                })

                # Download the image
                response = requests.get(thumb_url)
                if response.status_code == 200:
                    month_str = str(month).zfill(2) # Pad month with leading zero
                    filename = f"image_{year}_{month_str}.png"
                    filepath = os.path.join(new_path, filename) # Save in the new_path directory
                    with open(filepath, 'wb') as f:
                        f.write(response.content)
                    print(f"Saved {filename} to {new_path}")
                    break
                else:
                    print(f"Error downloading image for {year}-{month}: {response.status_code}")
            except ee.EEException as e:
                print(f"Earth Engine error for {year}-{month}: {e}")
        else:
            print(f"No suitable image found for {year}-{month}")

# Example usage (assuming basic_path is defined and exists)
# get_images(basic_path, 0, 47.01571746, 15.38990157, 2023) # Example for year 2023

### THIS IS SLOW IF THERE ARE MANY SUB-FOLDERS
# Get a list of sub-folders in basic_path and order them in increasing order
subfolders = [f.path for f in os.scandir(basic_path) if f.is_dir()]
subfolders.sort(key=lambda x: int(os.path.basename(x)))
# Get the last subfolder path
last_subfolder_path = subfolders[-1]
# Extract the last part of the path (the folder name) and convert it to an integer
last_index = int(os.path.basename(last_subfolder_path))
# Filter the data based on the last index
data = data[data['Grid_Max_Index'] >= last_index]

# FOR MANY SUB-FOLDERS, BETTER SET IS MANUALLY
data = data[data['Grid_Max_Index'] > 6156153]

# AND NOW RUN THE LOOP
for data_index in data['Grid_Max_Index']:
    index = data.index[data['Grid_Max_Index'] == data_index]
    try:
        get_images(basic_path, data_index, float(data['lat_WGS84'][index]),  float(data['lon_WGS84'][index]), 2021) # LAT LONG
    except:
        continue
os.chdir(basic_path)

In [ ]:
# Getting embeddings for full data may be problematic, as one has to loop through many sub-folders
# Most straight-forward option is get_embeddings function, but this may not work on some platforms like Google Drive, as it's slow with more than 10k sub-fodlers
# better option is function get_embeddings_no_moving_files, which may work if you make hierarchical structure of sub-folders
# even better is to get_embeddings_from_dict - i.e. make a dictionary of images (on a fast platform) and get embeddings from a single file
def get_embeddings_no_moving_files(image_folder):

    weights = ResNet18_Weights.IMAGENET1K_V1
    mean = weights.meta.get('mean', [0.485, 0.456, 0.406])
    std = weights.meta.get('std', [0.229, 0.224, 0.225]) # Corrected std value
    input_size = weights.meta.get('input_size', (3, 224, 224)) # Recommended input size for ImageNet
    transform = transforms.Compose([
        transforms.Resize((input_size[1], input_size[2])), # Use input_size from weights
        transforms.ToTensor(),        # Convert images to PyTorch tensors
        transforms.Normalize(mean=mean, std=std) # Use mean and std from weights
    ])

    temp_single_class_folder = os.path.join(image_folder, 'temp_single_class')
    os.makedirs(temp_single_class_folder, exist_ok=True)

    # Create a subdirectory within temp_single_class_folder to act as the class folder
    class_folder = os.path.join(temp_single_class_folder, 'all_images')
    os.makedirs(class_folder, exist_ok=True)

    valid_images = []
    for img_path in glob.glob(os.path.join(image_folder, '*.jpg')) + \
                    glob.glob(os.path.join(image_folder, '*.png')): # Add other extensions if needed
        if os.path.isfile(img_path):
            try:
                img = Image.open(img_path)
                img.verify() # Verify that it is an image
                # Move the image to the class_folder
                dest_path = os.path.join(class_folder, os.path.basename(img_path))
                if not os.path.exists(dest_path):
                  shutil.copy2(img_path, dest_path) # Use copy2 to preserve metadata
                  valid_images.append(dest_path)
            except (IOError, SyntaxError, UnidentifiedImageError):
                print(f"Skipping corrupted or unidentifiable image: {img_path}")
                continue


    dataset = ImageFolder(root=temp_single_class_folder, transform=transform) # Point ImageFolder to the temporary folder
    # Ensure that the dataset is not empty
    if len(dataset) == 0:
        print("No valid images found in the specified folder.")
        return None, None


    dataloader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=2)

    model = timm.create_model('resnet18', pretrained=True, num_classes=0) # num_classes=0 to remove the classifier head
    model.eval() # Set the model to evaluation mode

    # Extract embeddings
    embeddings = []
    image_paths = [] # Store corresponding image paths to associate embeddings

    with torch.no_grad(): # Disable gradient calculation
        for inputs, labels in dataloader:
            if torch.cuda.is_available():
                inputs = inputs.cuda()
                model.cuda()

            features = model(inputs) #
            embeddings.append(features.cpu().numpy())

            # Get the original image paths for the current batch
            # The indices in the batch correspond to the indices in dataset.samples
            batch_indices = dataloader.batch_sampler.sampler.indices if hasattr(dataloader.batch_sampler.sampler, 'indices') else list(range(len(dataset))) # Fallback for sequential sampler
            if hasattr(dataloader.batch_sampler, 'batch_size'):
                start_index = dataloader.batch_sampler.batch_size * (len(embeddings) - 1)
                current_batch_indices = batch_indices[start_index : start_index + inputs.size(0)]
            else:
                 current_batch_indices = list(range(len(dataset)))[dataloader.batch_sampler.offset:dataloader.batch_sampler.offset + inputs.size(0)] if hasattr(dataloader.batch_sampler, 'offset') else list(range(len(dataset)))[:inputs.size(0)] # Fallback for other samplers

            current_batch_paths = [dataset.samples[i][0] for i in current_batch_indices]
            image_paths.extend(current_batch_paths)


    # Concatenate embeddings from all batches
    embeddings = np.concatenate(embeddings, axis=0)

    # Clean up the temporary directory if it was created
    if 'temp_single_class_folder' in locals() and os.path.exists(temp_single_class_folder):
        shutil.rmtree(temp_single_class_folder)


    return embeddings, image_paths


# ###### COPY get_embeddings_from_dict  HERE!!!!!!!!!!!!!

In [ ]:
# Here test to get embeddings without moving or duplicating the files on
chunk = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Data_11_5_2025/full_data_WGS84_chunks/chunk_1.csv")

In [ ]:
def send_push_final(total_elapsed_time):
    token = 'am78agz8eyzewyo5xscrjngemouiwk'
    user = 'uar9sz3yciuvy67o4t4ctri7vwkvvh'
    conn = http.client.HTTPSConnection("api.pushover.net:443")
    conn.request("POST", "/1/messages.json",
      urllib.parse.urlencode({
        "token": token,
        "user": user,
        "title": "Job done - images downloaded",
        "message": f"Total elapsed time = {total_elapsed_time}",
      }), { "Content-type": "application/x-www-form-urlencoded" })
    conn.getresponse()

def format_timespan(seconds):
    """Formats a timespan in seconds into a human-readable string."""
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    return f"{int(hours)}h {int(minutes)}m {seconds:.2f}s"

In [ ]:
list_of_chunk_paths[0]

'/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Full_grid/Grid_Max/Chunk_1/541807'

In [ ]:
# This here does the followiong:
# implements function get_embeddings_no_moving_files in a loop for all sub-folders in destination_dir and save results as a dictionary,
# where the key is the sub-folder name and the value is the embeddings and image_paths

import os
import pickle

destination_dir = '/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Full_grid/Grid_Max/Chunk_1/' # Example destination directory
save_path = '/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Full_grid/Grid_Max/full_winter_summer_embeddings_chunk_1.pkl' # Example save path


total_start = time.time()

results_dict = {}

# Get a list of all sub-folders in the destination directory
list_of_chunk_paths = [os.path.join(destination_dir, str(index)) for index in chunk['Grid_Max_Index']]

# Optional: Sort subfolders by name if needed (e.g., numerically)
# all_subfolders.sort(key=lambda x: int(os.path.basename(x)))

# Iterate through each sub-folder

for i in range(len(list_of_chunk_paths)):
    image_folder = list_of_chunk_paths[i]
    # Extract the sub-folder name (which can be used as the dictionary key)
    subfolder_name = os.path.basename(image_folder)

    # Check if the image folder exists and is not empty
    if not os.path.exists(image_folder) or not os.listdir(image_folder):
        print(f"Image folder not found or is empty: {image_folder}. Skipping.")
        results_dict[subfolder_name] = {
            'embeddings': None,
            'image_paths': None # Or handle as needed for empty folders
        }
        continue # Skip to the next iteration if the folder doesn't exist or is empty

    # Call the get_embeddings_no_moving_files function
    embeddings, image_paths = get_embeddings_no_moving_files(image_folder)

    # Store results in the dictionary
    results_dict[subfolder_name] = {
        'embeddings': embeddings,
        'image_paths': image_paths
    }

    if embeddings is None or image_paths is None:
        print(f"Could not generate embeddings for folder {subfolder_name}")


    # Save the results_dict every 500 loops
    if (i + 1) % 500 == 0:
        with open(save_path, 'wb') as f:
            pickle.dump(results_dict, f)
        print(f"Saved partial results after processing {i + 1} folders to {save_path}")

# Save the final results dictionary after the loop finishes
with open(save_path, 'wb') as f:
    pickle.dump(results_dict, f)


print(f"Embeddings extraction complete. Results saved to {save_path}")

total_end = time.time()
total_elapsed_seconds = total_end - total_start
total_elapsed_time = format_timespan(total_elapsed_seconds)
print(f"Total elapsed time for extracting embeddings: {total_elapsed_time}")
# Optional: send a push notification
send_push_final(total_elapsed_time)

In [ ]:
image_folder

'/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Full_grid/Grid_Max/Test_100/680329'

In [ ]:
# TEST THE APPROACH

source_base_path = '/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Full_grid/Grid_Max/Chunk_1/'
destination_base_path = '/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Full_grid/Grid_Max/Test_100/'

# Ensure the destination base directory exists
os.makedirs(destination_base_path, exist_ok=True)

# Iterate through the grid indices in the test_data DataFrame
for grid_index in test_data['Grid_Max_Index']:
    # Construct the source folder path
    source_folder = os.path.join(source_base_path, str(grid_index))
    # Construct the destination folder path
    destination_folder = os.path.join(destination_base_path, str(grid_index))

    # Check if the source folder exists before attempting to copy
    if os.path.exists(source_folder) and os.path.isdir(source_folder):
        print(f"Copying subfolder: {source_folder} to {destination_folder}")
        try:
            # Use shutil.copytree to copy the folder and its contents
            # dirs_exist_ok=True is important to allow copying into an existing directory
            shutil.copytree(source_folder, destination_folder, dirs_exist_ok=True)
            print(f"Successfully copied {source_folder}")
        except Exception as e:
            print(f"Error copying folder {source_folder}: {e}")
    else:
        print(f"Source subfolder not found: {source_folder}. Skipping.")

print("Finished copying specified subfolders.")


In [ ]:
################ 14.7. -  FINAL VERSION
##### GET IMAGES
########################

In [ ]:
!pip install pystac_client odc.stac
!pip install torchgeo
!pip install humanfriendly

In [ ]:
from pystac_client import Client
from odc.stac import load
import geopy
from geopy.distance import geodesic
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images")  # set WD
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import numpy as np
from pystac_client import Client
from odc.stac import load
import shutil

import os
import tempfile

import time
import http.client
import urllib.parse

import timm
import torch
from lightning.pytorch import Trainer

from torchgeo.datamodules import EuroSAT100DataModule
from torchgeo.models import ResNet18_Weights
from torchgeo.trainers import ClassificationTask

import matplotlib.pyplot as plt # Import matplotlib.pyplot with the alias plt

import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.models import resnet50, ResNet50_Weights
# from torchvision.models import resnet18, ResNet18_Weights # Remove duplicate import
from torchgeo.models import ResNet18_Weights

import os
import numpy as np
import glob # Import glob for moving files
import timm # Import timm
import pandas as pd
from sklearn.cluster import KMeans
import pickle

from humanfriendly import format_timespan

import ee
import geopandas as gpd
from IPython.display import Image # Import the Image class
import requests
import datetime
import json

from torchvision.models.resnet import ResNet18_Weights
from PIL import Image, UnidentifiedImageError # Import Image and UnidentifiedImageError
from humanfriendly import format_timespan


from google.colab import drive
import pandas as pd

Mounted at /content/drive


In [ ]:
def get_images(basic_path, data_index, latitude, longitude, year):
    new_path = os.path.join(basic_path, str(data_index))
    # Check if directory exists before creating
    if not os.path.exists(new_path):
        os.makedirs(new_path)
    os.chdir(new_path)  # set WD

    point = ee.Geometry.Point(longitude, latitude)
    buffer = point.buffer(1000 / 2) # Create a square bbox by buffering by half the size

    for month in range(5, 9): # Loop through all months of the year
        start_date = ee.Date(f'{year}-{month}-01')
        # Calculate end date for the month, handling month lengths and leap years
        end_date = start_date.advance(1, 'month')

        # Define the Sentinel-2 image collection
        collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
            .filterDate(start_date, end_date) \
            .filterBounds(buffer) \
            .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10) \
            .map(lambda image: image.select(['B4', 'B3', 'B2']).divide(10000)) # Select RGB bands and scale

        # Get a single image (median composite) for the month
        image = collection.median()

        if image and image.bandNames().size().getInfo() > 0: # Check if an image was found and has bands
            # Define visualization parameters for True Color (RGB)
            vis_params_true_color = {
                'min': 0.0,
                'max': 0.3,
            }

            # Get the thumbnail URL
            try:
                thumb_url = image.getThumbUrl({
                    'bands': ['B4', 'B3', 'B2'],
                    'min': vis_params_true_color['min'],
                    'max': vis_params_true_color['max'],
                    'region': buffer.getInfo(),
                    'dimensions': 512
                })

                # Download the image
                response = requests.get(thumb_url)
                if response.status_code == 200:
                    month_str = str(month).zfill(2) # Pad month with leading zero
                    filename = f"image_{year}_{month_str}.png"
                    filepath = os.path.join(new_path, filename) # Save in the new_path directory
                    with open(filepath, 'wb') as f:
                        f.write(response.content)
                    print(f"Saved {filename} to {new_path}")
                    break
                else:
                    print(f"Error downloading image for {year}-{month}: {response.status_code}")
            except ee.EEException as e:
                print(f"Earth Engine error for {year}-{month}: {e}")
        else:
            print(f"No suitable image found for {year}-{month}")

    for month in (1,2,3,4,10,11,12): # Loop through all months of the year
        start_date = ee.Date(f'{year}-{month}-01')
        # Calculate end date for the month, handling month lengths and leap years
        end_date = start_date.advance(1, 'month')

        # Define the Sentinel-2 image collection
        collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
            .filterDate(start_date, end_date) \
            .filterBounds(buffer) \
            .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10) \
            .map(lambda image: image.select(['B4', 'B3', 'B2']).divide(10000)) # Select RGB bands and scale

        # Get a single image (median composite) for the month
        image = collection.median()

        if image and image.bandNames().size().getInfo() > 0: # Check if an image was found and has bands
            # Define visualization parameters for True Color (RGB)
            vis_params_true_color = {
                'min': 0.0,
                'max': 0.3,
            }

            # Get the thumbnail URL
            try:
                thumb_url = image.getThumbUrl({
                    'bands': ['B4', 'B3', 'B2'],
                    'min': vis_params_true_color['min'],
                    'max': vis_params_true_color['max'],
                    'region': buffer.getInfo(),
                    'dimensions': 512
                })

                # Download the image
                response = requests.get(thumb_url)
                if response.status_code == 200:
                    month_str = str(month).zfill(2) # Pad month with leading zero
                    filename = f"image_{year}_{month_str}.png"
                    filepath = os.path.join(new_path, filename) # Save in the new_path directory
                    with open(filepath, 'wb') as f:
                        f.write(response.content)
                    print(f"Saved {filename} to {new_path}")
                    break
                else:
                    print(f"Error downloading image for {year}-{month}: {response.status_code}")
            except ee.EEException as e:
                print(f"Earth Engine error for {year}-{month}: {e}")
        else:
            print(f"No suitable image found for {year}-{month}")

In [ ]:
chunk = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Data_11_5_2025/full_data_WGS84_chunks/chunk_1.csv")
chunk_top = chunk.head(10)

In [ ]:
root_folder = "/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Full_grid/Winter_Summer/Chunk_1_test/14_7_2025"

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Data_11_5_2025/full_data_WGS84_chunks/chunk_1.csv")
data_top = data.head(10)

basic_path = "/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Full_grid/Winter_Summer/Chunk_1_test/14_7_2025_1"

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='subtle-harmony-410811')

In [ ]:
data.to_csv("/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Full_grid/Winter_Summer/Chunk_1_test/14_7_2025_1/chunk_1.csv", index=False)

,lon_WGS84,lat_WGS84,Grid_Max_Index
0,28.646977,70.678009,541807
1,28.638899,70.669326,546754
2,28.690199,70.664431,546756
3,28.715840,70.661979,546757
4,29.922555,70.523217,556699
5,29.947885,70.520609,556700
6,29.914015,70.514592,561646
7,29.787207,70.499096,576483
8,29.719655,70.487000,586375
9,27.737725,70.658666,596192


In [ ]:
data = data_top
for data_index in data['Grid_Max_Index']:
    index = data.index[data['Grid_Max_Index'] == data_index]
    try:
        get_images(basic_path, data_index, float(data['lat_WGS84'][index]),  float(data['lon_WGS84'][index]), 2021) # LAT LONG
    except:
        continue
os.chdir(basic_path)

/tmp/ipython-input-4-2714659883.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  get_images(basic_path, data_index, float(data['lat_WGS84'][index]),  float(data['lon_WGS84'][index]), 2021) # LAT LONG


Saved image_2021_05.png to /content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Full_grid/Winter_Summer/Chunk_1_test/14_7_2025_1/541807
No suitable image found for 2021-1
No suitable image found for 2021-2
Saved image_2021_03.png to /content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Full_grid/Winter_Summer/Chunk_1_test/14_7_2025_1/541807


/tmp/ipython-input-4-2714659883.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  get_images(basic_path, data_index, float(data['lat_WGS84'][index]),  float(data['lon_WGS84'][index]), 2021) # LAT LONG


Saved image_2021_05.png to /content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Full_grid/Winter_Summer/Chunk_1_test/14_7_2025_1/546754
No suitable image found for 2021-1
No suitable image found for 2021-2
Saved image_2021_03.png to /content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Full_grid/Winter_Summer/Chunk_1_test/14_7_2025_1/546754


/tmp/ipython-input-4-2714659883.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  get_images(basic_path, data_index, float(data['lat_WGS84'][index]),  float(data['lon_WGS84'][index]), 2021) # LAT LONG


...

Saved image_2021_05.png to /content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Full_grid/Winter_Summer/Chunk_1_test/14_7_2025_1/576483
No suitable image found for 2021-1
No suitable image found for 2021-2
Saved image_2021_03.png to /content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Full_grid/Winter_Summer/Chunk_1_test/14_7_2025_1/576483


/tmp/ipython-input-4-2714659883.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  get_images(basic_path, data_index, float(data['lat_WGS84'][index]),  float(data['lon_WGS84'][index]), 2021) # LAT LONG


Saved image_2021_05.png to /content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Full_grid/Winter_Summer/Chunk_1_test/14_7_2025_1/586375
No suitable image found for 2021-1
No suitable image found for 2021-2
Saved image_2021_03.png to /content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Full_grid/Winter_Summer/Chunk_1_test/14_7_2025_1/586375


/tmp/ipython-input-4-2714659883.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  get_images(basic_path, data_index, float(data['lat_WGS84'][index]),  float(data['lon_WGS84'][index]), 2021) # LAT LONG


Saved image_2021_05.png to /content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Full_grid/Winter_Summer/Chunk_1_test/14_7_2025_1/596192
No suitable image found for 2021-1
No suitable image found for 2021-2
Saved image_2021_03.png to /content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Full_grid/Winter_Summer/Chunk_1_test/14_7_2025_1/596192


In [ ]:
# Ensure basic_path exists
os.makedirs(basic_path, exist_ok=True)
os.chdir(basic_path)

# Iterate through the data indices
for data_index_value in data_top['Grid_Max_Index']:
    # Create the main group folder based on data_index_value
    group_folder_name = f'group folder/{data_index_value}'
    group_folder_path = os.path.join(basic_path, group_folder_name)
    os.makedirs(group_folder_path, exist_ok=True)

    # Find the actual index in the data_top DataFrame
    try:
        index = data_top.index[data_top['Grid_Max_Index'] == data_index_value].tolist()[0]
    except IndexError:
        print(f"Warning: data_index {data_index_value} not found in data_top. Skipping.")
        continue # Skip to the next data_index_value if not found

    # Create 10 sub-folders within the group folder, named after the data_index
    for i in range(1, 11):
        sub_folder_name = str(data_index_value)
        sub_folder_path = os.path.join(group_folder_path, sub_folder_name)
        os.makedirs(sub_folder_path, exist_ok=True)

        # Call the get_images function to download images into this sub-folder
        # The get_images function will save images directly into the current working directory,
        # which is set to sub_folder_path within the function.
        try:
            # Pass the sub_folder_path to get_images
            get_images(sub_folder_path, data_index_value, float(data_top['lat_WGS84'][index]), float(data_top['lon_WGS84'][index]), 2021) # LAT LONG
        except Exception as e:
            print(f"Error getting images for data_index {data_index_value} in subfolder {sub_folder_path}: {e}")
            continue # Continue to the next sub-folder or data_index

# Change back to the initial basic_path after the loop
os.chdir(basic_path)


Saved image_2021_05.png to /content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Full_grid/Winter_Summer/Chunk_1_test/14_7_2025/group folder/541807/541807/541807
No suitable image found for 2021-1
No suitable image found for 2021-2
Saved image_2021_03.png to /content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Full_grid/Winter_Summer/Chunk_1_test/14_7_2025/group folder/541807/541807/541807
Saved image_2021_05.png to /content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Full_grid/Winter_Summer/Chunk_1_test/14_7_2025/group folder/541807/541807/541807
No suitable image found for 2021-1
No suitable image found for 2021-2
Saved image_2021_03.png to /content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Full_grid/Winter_Summer/Chunk_1_test/14_7_2025/group folder/541807/541807/541807
Saved image_2021_05.png to /content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Full_grid/Winter_Summer/Chunk_1_test/14_7_2025/group folder/541807/541807/541807
No suitable image found for 20

KeyboardInterrupt: 

In [ ]:
######### END   ###########

# Best pixel embeddings

In [ ]:
!pip install pystac_client odc.stac
!pip install torchgeo
!pip install humanfriendly
!pip install paramiko

In [ ]:
from pystac_client import Client
from odc.stac import load
import geopy
from geopy.distance import geodesic
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images")  # set WD
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import numpy as np
from pystac_client import Client
from odc.stac import load
import shutil

import os
import tempfile

import time
import http.client
import urllib.parse

import timm
import torch
from lightning.pytorch import Trainer

from torchgeo.datamodules import EuroSAT100DataModule
from torchgeo.models import ResNet18_Weights
from torchgeo.trainers import ClassificationTask

import matplotlib.pyplot as plt # Import matplotlib.pyplot with the alias plt

import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.models import resnet50, ResNet50_Weights
# from torchvision.models import resnet18, ResNet18_Weights # Remove duplicate import
from torchgeo.models import ResNet18_Weights

import numpy as np
import glob # Import glob for moving files
import timm # Import timm
import pandas as pd
from sklearn.cluster import KMeans
import pickle
from humanfriendly import format_timespan
from osgeo import gdal

import paramiko
import os

import rasterio
from rasterio.windows import Window
from rasterio.transform import from_bounds
import numpy as np
import matplotlib.pyplot as plt
from pyproj import Transformer

def send_push_final(total_elapsed_time):
    token = 'YYY'
    user = 'ZZZ'
    conn = http.client.HTTPSConnection("api.pushover.net:443")
    conn.request("POST", "/1/messages.json",
      urllib.parse.urlencode({
        "token": token,
        "user": user,
        "title": "Tiles found for demand",
        "message": f"Total elapsed time = {total_elapsed_time}",
      }), { "Content-type": "application/x-www-form-urlencoded" })
    conn.getresponse()


tiles = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Tiles_supply_GridMax.csv')
tiles = tiles.drop(columns=['Unnamed: 0'])

tiles = tiles.sort_values(by='tile_code').reset_index(drop=True) # order tiles by tile_code
# prompt: separate rows of tiles by tile_code

# Get unique tile codes
unique_tile_codes = tiles['tile_code'].unique()

# Initialize an empty dictionary to store DataFrames for each tile code
tiles_by_code = {}

# Iterate over unique tile codes and separate the rows
for code in unique_tile_codes:
  # Select rows where 'tile_code' matches the current code
  tiles_by_code[code] = tiles[tiles['tile_code'] == code].reset_index(drop=True)



def save_images_one_tile(image_path, one_tile, image_save_path):
    area_size_meters = 1000
    for i in  range(len(one_tile)):
        center_lon_wgs84 = one_tile['lon_WGS84'][i]
        center_lat_wgs84 = one_tile['lat_WGS84'][i]
        Grid_index = one_tile['Grid_index'][i]

        try:
            with rasterio.open(image_path) as src:
                # Get the image's Coordinate Reference System (CRS)
                image_crs = src.crs

                # Create a transformer to convert from WGS84 to the image's CRS
                transformer = Transformer.from_crs("EPSG:4326", image_crs, always_xy=True)

                # Transform the center WGS84 coordinates to the image's CRS
                center_x_image, center_y_image = transformer.transform(center_lon_wgs84, center_lat_wgs84)

                # Calculate the bounds of the 1 km x 1 km area in the image's CRS
                # The bounds will be centered around the transformed coordinates
                minx_image = center_x_image - area_size_meters / 2
                miny_image = center_y_image - area_size_meters / 2
                maxx_image = center_x_image + area_size_meters / 2
                maxy_image = center_y_image + area_size_meters / 2

                # Ensure the calculated bounds are within the image's bounds
                image_bounds = src.bounds
                minx_image = max(minx_image, image_bounds.left)
                miny_image = max(miny_image, image_bounds.bottom)
                maxx_image = min(maxx_image, image_bounds.right)
                maxy_image = min(maxy_image, image_bounds.top)

                # Convert the bounds in the image's CRS to pixel coordinates (window)
                # Using src.window requires coordinates in the image's CRS
                try:
                    window = src.window(minx_image, miny_image, maxx_image, maxy_image)

                    if window.width <= 0 or window.height <= 0:
                        print(f"Calculated window has zero or negative dimensions for Grid_index {Grid_index}. It might be outside the image extent.")
                    else:
                        # Read the data for the first three bands within the defined window
                        # Bands 1, 2, and 3 are typically Blue, Green, and Red for Sentinel-2
                        # Assuming 1: Blue, 2: Green, 3: Red. Adjust if your file is different.
                        # Read bands 1, 2, 3
                        # Ensure the window is an integer window for reading
                        window = window.round_lengths(op='floor').round_offsets(op='floor')

                        red = src.read(3, window=window)
                        green = src.read(2, window=window)
                        blue = src.read(1, window=window)


                        # Stack the bands in the desired order for plotting (RGB)
                        # For typical plotting with matplotlib's imshow, the order should be (Height, Width, Channels)
                        rgb_data = np.stack([red, green, blue], axis=0)

                        # Normalize the data for better visualization (optional)
                        # Adjust the scaling factor (e.g., 3000) based on your data's actual range
                        # A common approach for 16-bit data is to clip based on percentiles
                        # For simplicity, we'll use the previous clipping method.
                        # Check data type and range before scaling
                        if rgb_data.dtype in [np.uint16, np.int16, np.uint32, np.int32]:
                            # Example scaling for 16-bit or 32-bit data
                            # Clipping values between 0 and 3000 and scaling to 0-1
                            rgb_data = np.clip(rgb_data / 3000, 0, 1).astype(np.float32) # Scale to 0-1 and convert to float32
                        elif rgb_data.dtype in [np.uint8, np.int8]:
                            # Data is likely 8-bit, no scaling needed for 0-255 range
                            pass # Keep as is
                        else:
                            # For other data types, a simple scaling might be needed, or check the actual data range
                            # For now, assume it's already in a plottable range or needs a generic scaling
                            # As a fallback, a simple normalization if not already 0-1 or 0-255
                            if rgb_data.max() > 1.0: # Check if max value is greater than 1 (implies not 0-1 scaled)
                                rgb_data = rgb_data / rgb_data.max() # Simple normalization to 0-1


                        # Transpose the array to (Height, Width, Channels) for matplotlib
                        rgb_data = np.transpose(rgb_data, (1, 2, 0))

                        # Plot the image
                        fig, ax = plt.subplots(figsize=(8, 8))
                        ax.imshow(rgb_data)
                        ax.set_axis_off() # Hide axes for image display
                        fig.subplots_adjust(left=0, right=1, top=1, bottom=0) # Remove whitespace

                        # Specify the filename for saving the plot as JPG
                        # Consider making the filename more dynamic, perhaps incorporating the coordinates
                        output_filename = f'rgb_data_plot_Grid_index_{Grid_index}_lon_{center_lon_wgs84:.4f}_lat_{center_lat_wgs84:.4f}_1km.jpg'
                        image_save_path_with_file = os.path.join(image_save_path, output_filename)

                        # Save the figure as a JPG file
                        plt.savefig(image_save_path_with_file, bbox_inches='tight', pad_inches=0, format='jpg')
                        plt.close(fig) # Close the figure to free up memory
                        del image_save_path_with_file

                except ValueError as e:
                    print(f"Error creating window for Grid_index {Grid_index}: {e}")
                    print(f"Calculated bounds (Image CRS): minx={minx_image}, miny={miny_image}, maxx={maxx_image}, maxy={maxy_image}")
                    print(f"Image bounds (Image CRS): left={image_bounds.left}, bottom={image_bounds.bottom}, right={image_bounds.right}, top={image_bounds.top}")
                    # Handle the case where the target area is outside the image bounds
                    # You might want to skip plotting or handle this case appropriately

        except Exception as e:
            print(f"An error occurred while processing the image file: {e}")


image_save_path = '/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Supply/Best_pixel'

entries = os.listdir('/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Forest_Carbon_Monitoring/')

# Loop through all tiles
for tile_code in unique_tile_codes: # Iterate directly over unique_tile_codes # originally was unique_tile_codes
    one_tile = tiles_by_code[tile_code] # Access the dataframe using the tile_code
    if entries:
        # Filter entries based on the current tile_code
        filtered_entries = [entry.replace('- ', '') for entry in entries if f'{tile_code}' in entry]
        print(f"Filtered Entries containing '{tile_code}' with '- ' removed:")
        for entry in filtered_entries:
            print(entry)
            # Assuming there is only one relevant entry per tile_code for the image_path
            # If there can be multiple, you might need to adjust how image_path is determined
            image_path = f'/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Forest_Carbon_Monitoring/{entry}'
            save_images_one_tile(image_path, one_tile, image_save_path)
    else:
        print("No entries were listed from the remote directory.")


# make a list from all file names in '/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Supply/Best_pixel'
directory_path = '/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Supply/Best_pixel'
file_names = os.listdir(directory_path)

#  from file_names, keep just values after Grid_index_ and next _
extracted_values = []
for file_name in file_names:
    try:
        start_index = file_name.index('Grid_index_') + len('Grid_index_')
        end_index = file_name.index('_', start_index)
        extracted_value = int(file_name[start_index:end_index])
        extracted_values.append(extracted_value)
    except ValueError:
        # Handle cases where the expected pattern is not found
        print(f"Skipping file: {file_name} (Pattern 'Grid_index_' followed by '_' not found)")

# loop through entire tiles_by_code and find values in column Grid_index that are not in extracted_values

not_in_extracted = {}
for tile_code, df in tiles_by_code.items():
    grid_indices_in_df = set(df['Grid_index'].tolist())
    missing_indices = grid_indices_in_df - set(extracted_values)
    if missing_indices:
        not_in_extracted[tile_code] = list(missing_indices)

if not_in_extracted:
    print("Grid_index values in tiles_by_code not found in extracted_values:")
    for tile_code, missing_indices in not_in_extracted.items():
        print(f"Tile Code: {tile_code}")
        print(f"Missing Grid Indices: {missing_indices}")
else:
    print("All Grid_index values in tiles_by_code are present in extracted_values.")

keys_not_in_extracted = list(not_in_extracted.keys())


image_save_path = '/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Supply/Best_pixel'
entries = os.listdir('/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Forest_Carbon_Monitoring_1/')

# Loop through all tiles
for keys_not_in_extracted in not_in_extracted: # Iterate directly over unique_tile_codes
    one_tile = tiles_by_code[keys_not_in_extracted] # Access the dataframe using the tile_code
    if entries:
        # Filter entries based on the current tile_code
        filtered_entries = [entry.replace('- ', '') for entry in entries if f'{keys_not_in_extracted}' in entry]
        print(f"Filtered Entries containing '{keys_not_in_extracted}' with '- ' removed:")
        for entry in filtered_entries:
            print(entry)
            # Assuming there is only one relevant entry per tile_code for the image_path
            # If there can be multiple, you might need to adjust how image_path is determined
            image_path = f'/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Forest_Carbon_Monitoring/{entry}'
            save_images_one_tile(image_path, one_tile, image_save_path)
    else:
        print("No entries were listed from the remote directory.")


# Embeddings V2 without moving the files
import os
import glob
import shutil
from PIL import Image, UnidentifiedImageError # Import Image and UnidentifiedImageError
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import timm
from torchvision.models.resnet import ResNet18_Weights
import numpy as np

def get_embeddings(image_folder):

    weights = ResNet18_Weights.IMAGENET1K_V1
    mean = weights.meta.get('mean', [0.485, 0.456, 0.406])
    std = weights.meta.get('std', [0.229, 2.24, 2.25]) # Corrected std value
    input_size = weights.meta.get('input_size', (3, 224, 224)) # Recommended input size for ImageNet
    transform = transforms.Compose([
        transforms.Resize((input_size[1], input_size[2])), # Use input_size from weights
        transforms.ToTensor(),        # Convert images to PyTorch tensors
        transforms.Normalize(mean=mean, std=std) # Use mean and std from weights
    ])

    temp_single_class_folder = os.path.join(image_folder, 'temp_single_class')
    os.makedirs(temp_single_class_folder, exist_ok=True)

    # Create a subdirectory within temp_single_class_folder to act as the class folder
    class_folder = os.path.join(temp_single_class_folder, 'all_images')
    os.makedirs(class_folder, exist_ok=True)

    valid_images = []
    for img_path in glob.glob(os.path.join(image_folder, '*.jpg')) + \
                    glob.glob(os.path.join(image_folder, '*.png')): # Add other extensions if needed
        if os.path.isfile(img_path):
            try:
                img = Image.open(img_path)
                img.verify() # Verify that it is an image
                # Move the image to the class_folder
                dest_path = os.path.join(class_folder, os.path.basename(img_path))
                if not os.path.exists(dest_path):
                  shutil.copy2(img_path, dest_path) # Use copy2 to preserve metadata
                  valid_images.append(dest_path)
            except (IOError, SyntaxError, UnidentifiedImageError):
                print(f"Skipping corrupted or unidentifiable image: {img_path}")
                continue


    dataset = ImageFolder(root=temp_single_class_folder, transform=transform) # Point ImageFolder to the temporary folder
    # Ensure that the dataset is not empty
    if len(dataset) == 0:
        print("No valid images found in the specified folder.")
        return None, None


    dataloader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=2)

    model = timm.create_model('resnet18', pretrained=True, num_classes=0) # num_classes=0 to remove the classifier head
    model.eval() # Set the model to evaluation mode

    # Extract embeddings
    embeddings = []
    image_paths = [] # Store corresponding image paths to associate embeddings

    with torch.no_grad(): # Disable gradient calculation
        for inputs, labels in dataloader:
            if torch.cuda.is_available():
                inputs = inputs.cuda()
                model.cuda()

            features = model(inputs) #
            embeddings.append(features.cpu().numpy())

            # Get the original image paths for the current batch
            # The indices in the batch correspond to the indices in dataset.samples
            batch_indices = dataloader.batch_sampler.sampler.indices if hasattr(dataloader.batch_sampler.sampler, 'indices') else list(range(len(dataset))) # Fallback for sequential sampler
            if hasattr(dataloader.batch_sampler, 'batch_size'):
                start_index = dataloader.batch_sampler.batch_size * (len(embeddings) - 1)
                current_batch_indices = batch_indices[start_index : start_index + inputs.size(0)]
            else:
                 current_batch_indices = list(range(len(dataset)))[dataloader.batch_sampler.offset:dataloader.batch_sampler.offset + inputs.size(0)] if hasattr(dataloader.batch_sampler, 'offset') else list(range(len(dataset)))[:inputs.size(0)] # Fallback for other samplers

            current_batch_paths = [dataset.samples[i][0] for i in current_batch_indices]
            image_paths.extend(current_batch_paths)


    # Concatenate embeddings from all batches
    embeddings = np.concatenate(embeddings, axis=0)

    # Clean up the temporary directory if it was created
    if 'temp_single_class_folder' in locals() and os.path.exists(temp_single_class_folder):
        shutil.rmtree(temp_single_class_folder)


    return embeddings, image_paths

def get_grid_lat_lon_index(image_paths):
    cleaned_image_paths = [path.split('/')[-1] for path in image_paths]# in each element of  image_paths, remove all before last /
    grid_indices = [path.split('Grid_index_')[-1].split('_')[0] for path in cleaned_image_paths]

    lon_values = []
    for path in cleaned_image_paths:
        # Find the index of 'lon_'
        lon_index = path.find('lon_')
        if lon_index != -1:
            # Find the index of the next '_' after 'lon_'
            start_index = lon_index + len('lon_')
            end_index = path.find('_', start_index)
            if end_index != -1:
                # Extract the substring between 'lon_' and the next '_'
                lon_value_str = path[start_index:end_index]
                try:
                    # Convert the extracted string to a float and add to the list
                    lon_value = float(lon_value_str)
                    lon_values.append(lon_value)
                except ValueError:
                    print(f"Could not convert {lon_value_str} to float in path: {path}")
            else:
                print(f"Could not find the '_' after 'lon_' in path: {path}")
        else:
            print(f"Could not find 'lon_' in path: {path}")

    # Print the extracted longitude values
    print(lon_values)

    # Similarly, extract latitude values
    lat_values = []
    for path in cleaned_image_paths:
        # Find the index of 'lat_'
        lat_index = path.find('lat_')
        if lat_index != -1:
            # Find the index of the next '_' after 'lat_'
            start_index = lat_index + len('lat_')
            end_index = path.find('_', start_index)
            if end_index != -1:
                # Extract the substring between 'lat_' and the next '_'
                lat_value_str = path[start_index:end_index]
                try:
                    # Convert the extracted string to a float and add to the list
                    lat_value = float(lat_value_str)
                    lat_values.append(lat_value)
                except ValueError:
                    print(f"Could not convert {lat_value_str} to float in path: {path}")
            else:
                print(f"Could not find the '_' after 'lat_' in path: {path}")
        else:
            print(f"Could not find 'lat_' in path: {path}")
    return grid_indices, lon_values, lat_values


# make a single data frame, where columns are grid_indices, lon_values, lat_values and  embeddings. Each embedding is its own column

# Create a dictionary to store the data
data = {
    'grid_indices': grid_indices,
    'lon_values_WGS84': lon_values,
    'lat_values_WGS84': lat_values
}

# Add each embedding dimension as a separate column
for i in range(embeddings.shape[1]):
    data[f'embedding_{i}'] = embeddings[:, i]

# Create the DataFrame
embeddings_df = pd.DataFrame(data)


# Load the data
CH_data_demand = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Data_11_5_2025/CH_SINC_foresters_supply_demand_GRID_max_WGS84.csv')
CH_data_demand = CH_data_demand.drop(columns=['Unnamed: 0'])
CH_data_demand = CH_data_demand.dropna(subset=['grid_abelow_biom'])
CH_data_demand['Grid_index'] = CH_data_demand.index
CH_data_demand = CH_data_demand.reset_index(drop=True)


# https://carpentries-incubator.github.io/geospatial-python/instructor/05-access-data.html
# GET TILE CODES BY COORDINATES

api_url = "https://earth-search.aws.element84.com/v1"

from pystac_client import Client

client = Client.open(api_url)
collections = client.get_collections()
collection_sentinel_2_l2a = "sentinel-2-l2a"
from shapely.geometry import Point

#  find the latest tile for a given point
def find_latest_tile_for_point(client, collection_name, point):
    search = client.search(
        collections=[collection_name],
        intersects=point,
        sortby=[{"field": "properties.datetime", "direction": "desc"}]  # Sort by datetime descending
    )

    # Iterate through the items and return the first one (which is the latest due to sorting)
    for item in search.items():  # Use .items() instead of .get_items() (deprecated)
        return item

    # If no items are found for the point
    return None

total_start = time.time()

# Initialize an empty list to store row data
rows_list = []

# X typically represents longitude, and Y represents latitude # survey_x_coo	survey_y_coo



for i in range(len(CH_data_demand)):
    # Extract the coordinates from the DataFrame.
    longitude = CH_data_demand.loc[i, 'lon_WGS84'] # LONGITUDE in WGS84
    latitude = CH_data_demand.loc[i, 'lat_WGS84'] # LATITUDE in WGS84
    Grid_index = CH_data_demand.loc[i, 'Grid_index']
    survey_x_coo = CH_data_demand.loc[i, 'survey_x_coo']
    survey_y_coo = CH_data_demand.loc[i, 'survey_y_coo']

    sample_point = {
        "type": "Point",
        "coordinates": [longitude, latitude] # LONGITUDE AND LATITUDE
    }

    # Find the latest tile for the sample point
    latest_tile = find_latest_tile_for_point(client, collection_sentinel_2_l2a, sample_point)

    if latest_tile:
        # print("Found the latest tile:")
        # print(f"Item ID: {latest_tile.id}")
        # print(f"Datetime: {latest_tile.properties['datetime']}")
        parts = latest_tile.id.split('_')
        extracted_text = parts[1] if len(parts) >= 3 else 'XXXX'
    else:
        # print("No tiles found for the specified point.")
        extracted_text = 'XXXX'

    # Append the results as a dictionary to the list
    rows_list.append({'Grid_index': Grid_index, 'lon_WGS84': longitude, 'lat_WGS84': latitude, 'survey_x_coo': survey_x_coo, 'survey_y_coo': survey_y_coo, 'tile_code': extracted_text})

# Create the DataFrame from the list of dictionaries after the loop
tiles = pd.DataFrame(rows_list)

total_end = time.time()
total_elapsed_seconds = total_end - total_start
# Assuming format_timespan and send_push_final are defined elsewhere
total_elapsed_time = format_timespan(total_elapsed_seconds)
send_push_final(total_elapsed_time)

tiles.to_csv('/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Tiles_supply_GridMax.csv')

display(tiles.head()) # Display the head of the resulting DataFrame

full_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Data_11_5_2025/full_data_part_1.csv')

def find_latest_tile_for_point(client, collection_name, point):
    search = client.search(
        collections=[collection_name],
        intersects=point,
        sortby=[{"field": "properties.datetime", "direction": "desc"}]  # Sort by datetime descending
    )

    # Iterate through the items and return the first one (which is the latest due to sorting)
    for item in search.items():  # Use .items() instead of .get_items() (deprecated)
        return item

    # If no items are found for the point
    return None


api_url = "https://earth-search.aws.element84.com/v1"

from pystac_client import Client

client = Client.open(api_url)
collections = client.get_collections()
collection_sentinel_2_l2a = "sentinel-2-l2a"
from shapely.geometry import Point

rows_list = []

for i in range(len(full_data)):
    # Extract the coordinates from the DataFrame.
    longitude = full_data.loc[i, 'lon_WGS84'] # LONGITUDE in WGS84
    latitude = full_data.loc[i, 'lat_WGS84'] # LATITUDE in WGS84
    Grid_index = full_data.loc[i, 'Grid_index']
    survey_x_coo = full_data.loc[i, 'lon']
    survey_y_coo = full_data.loc[i, 'lat']

    sample_point = {
        "type": "Point",
        "coordinates": [longitude, latitude] # LONGITUDE AND LATITUDE
    }

    # Find the latest tile for the sample point
    latest_tile = find_latest_tile_for_point(client, collection_sentinel_2_l2a, sample_point)

    if latest_tile:
        # print("Found the latest tile:")
        # print(f"Item ID: {latest_tile.id}")
        # print(f"Datetime: {latest_tile.properties['datetime']}")
        parts = latest_tile.id.split('_')
        extracted_text = parts[1] if len(parts) >= 3 else 'XXXX'
    else:
        # print("No tiles found for the specified point.")
        extracted_text = 'XXXX'

    # Append the results as a dictionary to the list
    rows_list.append({'Grid_index': Grid_index, 'lon_WGS84': longitude, 'lat_WGS84': latitude, 'lon': survey_x_coo, 'lat': survey_y_coo, 'tile_code': extracted_text})

    # Save the DataFrame every 50000 iterations
    if (i + 1) % 50000 == 0:
        tiles = pd.DataFrame(rows_list)
        # Define the save path (adjust as needed, perhaps add a counter or timestamp)
        save_path = '/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Tiles_full_GridMax_1.csv'
        tiles.to_csv(save_path, index=False)
        print(f"Saved progress at iteration {i + 1} to {save_path}")
        # Optionally, clear the rows_list after saving to manage memory,
        # but you'll need to concatenate saved parts later.
        # rows_list = []


# Create the DataFrame from the remaining list of dictionaries after the loop
if rows_list: # Check if there are any remaining rows
    tiles = pd.DataFrame(rows_list)
    # Define the final save path
    final_save_path = '/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Tiles_full_GridMax_1.csv'
    tiles.to_csv(final_save_path, index=False)
    print(f"Saved final results to {final_save_path}")


total_end = time.time()
total_elapsed_seconds = total_end - total_start
# Assuming format_timespan and send_push_final are defined elsewhere
total_elapsed_time = format_timespan(total_elapsed_seconds)
send_push_final(total_elapsed_time)

#  find the latest tile for a given point
def find_latest_tile_for_point(client, collection_name, point):
    search = client.search(
        collections=[collection_name],
        intersects=point,
        sortby=[{"field": "properties.datetime", "direction": "desc"}]  # Sort by datetime descending
    )

    # Iterate through the items and return the first one (which is the latest due to sorting)
    for item in search.items():  # Use .items() instead of .get_items() (deprecated)
        return item

    # If no items are found for the point
    return None

total_start = time.time()

# Initialize an empty list to store row data
rows_list = []

# X typically represents longitude, and Y represents latitude # survey_x_coo	survey_y_coo


for i in range(len(full_data)):
    # Extract the coordinates from the DataFrame.
    longitude = full_data.loc[i, 'lon_WGS84'] # LONGITUDE in WGS84
    latitude = full_data.loc[i, 'lat_WGS84'] # LATITUDE in WGS84
    Grid_index = full_data.loc[i, 'Grid_index']
    survey_x_coo = full_data.loc[i, 'lon']
    survey_y_coo = full_data.loc[i, 'lat']

    sample_point = {
        "type": "Point",
        "coordinates": [longitude, latitude] # LONGITUDE AND LATITUDE
    }

    # Find the latest tile for the sample point
    latest_tile = find_latest_tile_for_point(client, collection_sentinel_2_l2a, sample_point)

    if latest_tile:
        # print("Found the latest tile:")
        # print(f"Item ID: {latest_tile.id}")
        # print(f"Datetime: {latest_tile.properties['datetime']}")
        parts = latest_tile.id.split('_')
        extracted_text = parts[1] if len(parts) >= 3 else 'XXXX'
    else:
        # print("No tiles found for the specified point.")
        extracted_text = 'XXXX'

    # Append the results as a dictionary to the list
    rows_list.append({'Grid_index': Grid_index, 'lon_WGS84': longitude, 'lat_WGS84': latitude, 'lon': survey_x_coo, 'lat': survey_y_coo, 'tile_code': extracted_text})


# Create the DataFrame from the list of dictionaries after the loop
tiles = pd.DataFrame(rows_list)

total_end = time.time()
total_elapsed_seconds = total_end - total_start
# Assuming format_timespan and send_push_final are defined elsewhere
total_elapsed_time = format_timespan(total_elapsed_seconds)
send_push_final(total_elapsed_time)

tiles.to_csv('/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Tiles_full_GridMax_1.csv')

display(tiles.head()) # Display the head of the resulting DataFrame

# list folders in sftp

# Function to list contents of a remote directory
def list_remote_directory(sftp, remote_dir):
    try:
        print(f"Listing contents of remote directory: {remote_dir}")
        entries = sftp.listdir_attr(remote_dir)
        print("Contents:")
        for entry in entries:
            print(f"- {entry.filename}")
        return [entry.filename for entry in entries]
    except FileNotFoundError:
        print(f"Error: Remote directory not found at {remote_dir}")
        return None
    except Exception as e:
        print(f"Error listing directory {remote_dir}: {e}")
        return entries

# Remote directory you want to list
remote_directory_to_list = '/Sentinel-2_composite_mosaic_2021' # <-- **CHANGE THIS** to the actual path you want to list

hostname = '64.225.134.56'
port = 22
username = 'YYY' # ask for access here https://www.forestcarbonplatform.org/platform/
password = 'ZZZ'


try:
    # Re-create an SSH client and connect (assuming the previous connection might have been closed)
    # Or reuse the 'client' and 'sftp' objects if the connection is still active
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    print(f"Connecting to {hostname}:{port}...")
    client.connect(hostname, port, username, password)
    print("Connected successfully.")

    sftp = client.open_sftp()
    print("SFTP session opened.")

    # List the contents of the specified remote directory
    entries = list_remote_directory(sftp, remote_directory_to_list)

except paramiko.AuthenticationException:
    print("Authentication failed. Please check your username and password.")
except paramiko.SSHException as sshEx:
    print(f"SSH error: {sshEx}")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Close the SFTP session and SSH connection
    if 'sftp' in locals() and sftp:
        sftp.close()
        print("SFTP session closed.")
    if 'client' in locals() and client:
        client.close()
        print("SSH connection closed.")


if entries:
  # Filter entries to include only those containing any of the unique_tile_codes
  # and remove the '- ' prefix
  filtered_entries = [
      entry.replace('- ', '') for entry in entries
      if any(code in entry for code in unique_tile_codes)
  ]

  if filtered_entries:
    print("Filtered Entries matching unique tile codes with '- ' removed:")
    for entry in filtered_entries:
      print(entry)
  else:
    print("No entries matching the unique tile codes were found in the remote directory.")
else:
  print("No entries were listed from the remote directory.")

  # make a list of files that are in local_dir = '/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Forest_Carbon_Monitoring'
local_dir = '/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Forest_Carbon_Monitoring'
local_files = os.listdir(local_dir)
# from filtered_entries, make a list of those that are not in local_files
not_in_local = [entry for entry in filtered_entries if entry not in local_files]


# list all files in tiles
entries = os.listdir('/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Forest_Carbon_Monitoring/')
# from entries, keep only values until first _
processed_entries = [entry.split('_', 1)[0] for entry in entries]
# make list of values that are in processed_entries but not in unique_tile_codes
# These are tiles needed for demand images, but not for supply
processed_entries_set = set(processed_entries)
unique_tile_codes_set = set(unique_tile_codes)
not_needed_for_not_supply = list(processed_entries_set - unique_tile_codes_set)
# From folder with tiles, remove all tiles not needed for supply data
image_dir = '/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Forest_Carbon_Monitoring/'
all_files = os.listdir(image_dir)
for file_name in all_files:
  for prefix in not_needed_for_not_supply:
    if file_name.startswith(prefix):
      file_path = os.path.join(image_dir, file_name)
      try:
        os.remove(file_path)
        print(f"Removed file: {file_name}")
      except OSError as e:
        print(f"Error removing file {file_name}: {e}")
      break
#  make a list of entries that are in unique_tile_codes but not in processed_entries
needed_for_supply = list(unique_tile_codes_set - processed_entries_set)
# make a list of values from filtered_entries which start with entries in needed_for_supply
needed_files_for_supply = [
    entry for entry in filtered_entries if entry.split('_', 1)[0] in needed_for_supply]


# list all entries that start with values in unique_tile_codes

if entries:
  # Filter entries to include only those containing any of the unique_tile_codes
  # and remove the '- ' prefix
  filtered_entries = [
      entry.replace('- ', '') for entry in entries
      if any(entry.startswith(code) for code in unique_tile_codes)
  ]

  if filtered_entries:
    print("Filtered Entries matching unique tile codes with '- ' removed:")
    for entry in filtered_entries:
      print(entry)
  else:
    print("No entries matching the unique tile codes were found in the remote directory.")
else:
  print("No entries were listed from the remote directory.")


# SSH connection details - DOWNLOAD TILES
hostname = '64.225.134.56'
port = 22
username = 'YY' # https://www.forestcarbonplatform.org/platform/ here you can ask for access
password = 'ZZZ'

# Local directory to save files
local_dir = '/content/drive/MyDrive/Colab Notebooks/SINCERE_S_D/Images/Forest_Carbon_Monitoring'
os.makedirs(local_dir, exist_ok=True)

for entry in filtered_entries: # needed_files_for_supply
    # Remote file or directory you want to download
    remote_path = f'/Sentinel-2_composite_mosaic_2021/{entry}' # <-- **CHANGE THIS** to the actual path on the server

    try:
        # Create an SSH client
        client = paramiko.SSHClient()
        client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

        # Connect to the server
        print(f"Connecting to {hostname}:{port}...")
        client.connect(hostname, port, username, password)
        print("Connected successfully.")

        # Open an SFTP session
        sftp = client.open_sftp()
        print("SFTP session opened.")

        # Function to download a file
        def download_file(sftp, remote_path, local_path):
            try:
                print(f"Downloading {remote_path} to {local_path}...")
                sftp.get(remote_path, local_path)
                print("Download complete.")
            except FileNotFoundError:
                print(f"Error: Remote file not found at {remote_path}")
            except Exception as e:
                print(f"Error downloading file: {e}")

        # Function to download a directory recursively
        def download_directory(sftp, remote_dir, local_dir):
            try:
                print(f"Downloading directory {remote_dir} to {local_dir}...")
                os.makedirs(local_dir, exist_ok=True)
                for entry in sftp.listdir_attr(remote_dir):
                    remote_entry_path = os.path.join(remote_dir, entry.filename).replace("\\", "/") # Handle path separators
                    local_entry_path = os.path.join(local_dir, entry.filename)

                    if entry.st_mode & 0o40000: # Check if it's a directory
                        download_directory(sftp, remote_entry_path, local_entry_path)
                    else: # It's a file
                        download_file(sftp, remote_entry_path, local_entry_path)
                print(f"Directory download complete for {remote_dir}.")
            except FileNotFoundError:
                print(f"Error: Remote directory not found at {remote_dir}")
            except Exception as e:
                print(f"Error downloading directory {remote_dir}: {e}")


        # Check if the remote_path is a file or directory and download accordingly
        try:
            remote_stat = sftp.stat(remote_path)
            if remote_stat.st_mode & 0o40000: # It's a directory
                # Define the local directory name based on the remote directory name
                local_target_path = os.path.join(local_dir, os.path.basename(remote_path))
                download_directory(sftp, remote_path, local_target_path)
            else: # It's a file
                # Define the local file path based on the remote file name
                local_target_path = os.path.join(local_dir, os.path.basename(remote_path))
                download_file(sftp, remote_path, local_target_path)
        except FileNotFoundError:
            print(f"Error: Remote path not found at {remote_path}")
        except Exception as e:
            print(f"Error checking remote path type: {e}")


    except paramiko.AuthenticationException:
        print("Authentication failed. Please check your username and password.")
    except paramiko.SSHException as sshEx:
        print(f"SSH error: {sshEx}")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Close the SFTP session and SSH connection
        if 'sftp' in locals() and sftp:
            sftp.close()
            print("SFTP session closed.")
        if 'client' in locals() and client:
            client.close()
            print("SSH connection closed.")